# Custom Reviewer Workflow
This notebook demonstrates how to create a custom workflow using the `AgentWorkflow` class.


# Imports

In [1]:
from dotenv import load_dotenv
from typing import Any, Dict, List

from saw.core.model_interface import amodel_call
from saw.workflow import AgentWorkflow

In [2]:
# Load environment variables
load_dotenv("../docker/.env")

True

# Define Custom Reviewer Function
Define a custom function that will be used by the `AgentWorkflow` to review content.


In [3]:
async def custom_reviewer(prompts: List[Dict[str, Any]]) -> str:
    reviews = []
    for prompt in prompts:
        response = await amodel_call(
            prompt=prompt["prompt"],
            provider=prompt["provider"],
            model=prompt["model"],
            system_prompt=prompt["system_prompt"],
        )
        print(f"{prompt["prompt"]}: {response}")
        reviews.append(response)

    summary_prompt = "Summarize the following reviews into a single insight:\n" + "\n".join(reviews)
    summary = await amodel_call(
        prompt=summary_prompt,
        provider=prompts[0]["provider"],
        model=prompts[0]["model"],
        system_prompt="You are a helpful assistant.",
    )
    print(f"Summary: {summary}")
    return summary

# Initialize Agent Workflow

In [4]:
# Initialize the `AgentWorkflow` with the custom reviewer function.
agent = AgentWorkflow(operation="custom", custom_workflow=custom_reviewer)

In [5]:
# Define Query and Prompts
prompt_list = [
    {
        "prompt": "Review the new smartphone model from a technical perspective.",
        "model": "gemini-2.0-flash",
        "provider": "google",
        "system_prompt": "You are a technical expert.",
        "functions": [str.lower]
    },
    {
        "prompt": "Review the new smartphone model from a user experience perspective.",
        "model": "deepseek-r1:1.5b",
        "provider": "ollama",
        "system_prompt": "You are a user experience expert.",
        "functions": [str.lower]
    },
    {
        "prompt": "Review the new smartphone model from a market perspective.",
        "model": "llama3-8b-8192",
        "provider": "groq",
        "system_prompt": "You are a market analyst.",
        "functions": [str.lower]
    }
]

In [6]:
# Execute the workflow
result = await agent.execute(prompts=prompt_list, async_mode=True)

Function Arguments: {'prompts': [{'prompt': 'Review the new smartphone model from a technical perspective.', 'model': 'gemini-2.0-flash', 'provider': 'google', 'system_prompt': 'You are a technical expert.', 'functions': [<method 'lower' of 'str' objects>]}, {'prompt': 'Review the new smartphone model from a user experience perspective.', 'model': 'deepseek-r1:1.5b', 'provider': 'ollama', 'system_prompt': 'You are a user experience expert.', 'functions': [<method 'lower' of 'str' objects>]}, {'prompt': 'Review the new smartphone model from a market perspective.', 'model': 'llama3-8b-8192', 'provider': 'groq', 'system_prompt': 'You are a market analyst.', 'functions': [<method 'lower' of 'str' objects>]}]}
Response: dict_keys(['candidates', 'create_time', 'response_id', 'model_version', 'prompt_feedback', 'usage_metadata', 'automatic_function_calling_history', 'parsed'])
Model: gemini-2.0-flash
Usage: cached_content_token_count=None candidates_token_count=1814 prompt_token_count=16 tota

In [7]:
print(result)

The "Optimus X" smartphone from XYZ Inc. is positioned to compete in the high-end market by leveraging its superior camera system, long battery life, and strategic partnerships. A balanced pricing and marketing strategy focusing on influencers, online advertising, and in-store promotions will be key to capturing market share and establishing the Optimus X as a leader in the premium smartphone segment.

